In [1]:
import torch
torch.cuda.is_available()

True

In [3]:
import ollama
ollama.list()

{'models': [{'name': 'llama3:8b',
   'model': 'llama3:8b',
   'modified_at': '2024-04-25T11:46:48.7619245+09:00',
   'size': 4661224578,
   'digest': 'a6990ed6be412c6a217614b0ec8e9cd6800a743d5dd7e1d7fbe9df09e61d5615',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': ['llama'],
    'parameter_size': '8B',
    'quantization_level': 'Q4_0'}}]}

In [3]:
import pandas as pd

file_path = "./prepro_company_data.csv"
df = pd.read_csv(file_path)
df["Positive_Score"] = None
df

,company,code,Date,Title,Positive_Score
0,삼성전자,'005930,2021-03-12,진짜 5 G 28 서비스 활성화 첫 발 과기부 TF 발족 회의,None
1,삼성전자,'005930,2021-03-05,아버님 댁 에어 프라이어 놓다 드리다,None
2,삼성전자,'005930,2021-05-14,ESG 경영 첫 단추 늘다 여성 관리자 키우다,None
3,삼성전자,'005930,2021-02-05,삼성 공장 증설 19조 투자 주 정부 9000억 세제 요청,None
4,삼성전자,'005930,2021-01-22,화웨이 서다 분리 중 가폰 어너 인텔 삼성 반도체 계약,None
...,...,...,...,...,...
15895,코오롱인더,'120110,2023-07-21,연구개발 1.1조 쏟다 코오롱 인 더 하반기 과실 딴다,None
15896,코오롱인더,'120110,2023-10-20,코오롱 인 더 3 Q 영업 익 시장 기 대치 회 4 Q 패션 성 수기 효과 기대 IBK,None
15897,코오롱인더,'120110,2023-10-27,코오롱 몰다 파격 할인 2023 OLO 모드 K 쇼핑 위 크다 진행,None
15898,코오롱인더,'120110,2023-12-01,추다 날씨 아우 터 판매 확대 코오롱 인 더 실적 개선 전망 IBK,None


In [ ]:
import pandas as pd
import re
import os
import ollama
import time

# CSV 파일 경로
file_path = "./LLAMA3-8B.csv"

# 파일 존재 여부 확인 및 초기화
if os.path.exists(file_path):
    df = pd.read_csv(file_path)
else:
    df = pd.DataFrame(columns=['Title', 'company', 'Positive_Score'])  # 기본적인 컬럼을 가진 빈 DataFrame 생성
    df.to_csv(file_path, index=False, encoding='utf-8-sig')  # 빈 파일을 생성하여 저장

model = "llama3:8b"

# 시스템 명령 정의
system_instruction = "금융전문가로 가정해주세요. 당신은 주식 추천 경험이 있는 금융 전문가입니다."

# 감성 분석 및 점수 저장
while True:
    try:
        for idx, row in df.iterrows():
            title = row['Title']
            company = row['company']
            
            # 'Positive_Score' 값이 이미 있는 경우 건너뛰기
            if pd.notnull(row['Positive_Score']):
                continue
            
            print(f"Processing headline {idx}: {title}")
            
            # 유저 프롬프트 생성
            user_prompt = f"헤드라인: {title}. 좋은 뉴스, 나쁜 뉴스, 불확실한 뉴스 세 가지가 될 확률이 합쳐서 1일 때 좋은 뉴스일 정도를 확률로만 답변해주세요. (예시답변: 0.2) {company}에 좋은 영향을 숫자로만 답해주세요"
            
            # ollama 모델을 사용하여 응답 받기
            response = ollama.chat(model=model, messages=[
                {"role": "system", "content": system_instruction},  # 시스템 명령 추가
                {"role": "user", "content": user_prompt}
            ])
            
            # 응답에서 숫자만 추출하여 저장
            score = re.findall(r'\d+\.\d+', response['message']['content'])
            if score:
                df.at[idx, 'Positive_Score'] = float(score[0])
                df.to_csv(file_path, index=False, encoding='utf-8-sig')  # 점수를 CSV 파일에 저장
            else:
                print(f"No score found for headline {idx}")
        
        break  # 에러 없이 완료되었으면 반복문 종료
    
    except Exception as e:
        print(f"An error occurred: {str(e)}. Retrying in 1 minute...")
        time.sleep(60)  # 1분 대기 후 다시 시도

print("감성 분석이 완료되었습니다.")